## Логическая модель данных:

<img src="https://github.com/timoti1/T-SQL/blob/master/SQL/img/SwimmersDB.png?raw=1" />

<b>Пример "плохого" DDL-кода:</b>

In [ ]:
create table Category (
CategoryID int identity primary key,
Name nvarchar(20) not null UNIQUE, 
ModifiedDate date not null
)

create table SwimmingClub (
SwimmingClubID int identity primary key,
Name varchar(50) not null,
City varchar(50) not null,
Address varchar(50) not NULL,
Phone  varchar(50) not null,
YearEstablished int not null,
ModifiedDate date not NULL
)

create table Swimmer (
SwimmerID int identity primary key,
SwimmingClubID int not null, 
CategoryId int not null references Category(CategoryId),
FirstName varchar(50) not null,
LastName varchar(50) not null,
YearOfBirth int not NULL,
Gender int not null,
ModifiedDate date not NULL
)

create table Coach (
CoachID int identity primary key,
FirstName varchar(50) not null,
LastName varchar(50) not null,
DateOfBirth date not NULL,
CategoryID int not null references Category(CategoryID),
ModifiedDate date not NULL
)

create table SwimmerCoach (
SwimmerID int not null references Swimmer(SwimmerID),
CoachID int not null references Coach(CoachID), 
ModifiedDate date not NULL
)
go

<b>"Хороший" DDL-код подразумевает:</b>
- следование определенным naming conventions и code style
- готовность к последущему рефакторингу
- готовность к миграции на другие СУБД или downgrade текущей
- воспроизводимость результатов при перезапуске
- читаемость
- ...


Файл <b>"SQL\code\DDL\tempdb - DDL - table - dbo.Category.sql"</b>:

In [2]:
--выбираем активную БД
use tempdb
go

--создаем объект, предварительно проверяя был ли он создан ранее
--стараемся использовать "классический" подход

--drop table if exists #Category
if OBJECT_ID('dbo.Category', 'U') is not null
   drop table dbo.Category
go

create table dbo.Category 
(
    CategoryID      tinyint         not null    identity,
    [Name]          nvarchar(20)    not null, 
    ModifiedDate    datetime        not null,

    constraint PK_Category primary key (
        CategoryID
    )
)
go

Файл <b>"SQL\code\DDL\tempdb - DDL - constraint - dbo.Category.sql"</b>:

In [2]:
--выбираем активную БД
use tempdb
go

--удаляем констрейнт если он был создан ранее
--AK_Category_Name
if exists(select 1 from INFORMATION_SCHEMA.TABLE_CONSTRAINTS where CONSTRAINT_NAME = 'AK_Category_Name')
   alter table dbo.Category drop constraint AK_Category_Name
go

alter table dbo.Category add constraint AK_Category_Name unique 
(
    [Name]
)
go

--DF_Category_ModifiedDate. Внимание, подход меняется!
--if exists(select 1 from INFORMATION_SCHEMA.TABLE_CONSTRAINTS where CONSTRAINT_NAME = 'DF_Category_ModifiedDate')
if OBJECT_ID('DF_Category_ModifiedDate', 'D') is not null
   alter table dbo.Category drop constraint DF_Category_ModifiedDate
go

alter table dbo.Category add constraint DF_Category_ModifiedDate default GETDATE() for ModifiedDate
go

Остальные скрипты DDL находятся в директории
<b>SQL\code\DDL\ </b>

Пример хранимой процедуры:

In [3]:
use tempdb
go

if OBJECT_ID('dbo.usp_GetSwimmersList', 'P') is not null
   drop procedure dbo.usp_GetSwimmersList
go

-------------------------------------------------------------------------------------------
-- procedure returns list of Swimmers from given clubs
-- created by:   Timofey Gavrilenko
-- created date: 4/22/2019
-- sample call:  
-- exec dbo.usp_GetSwimmersList @parameters = N'[{"Club": "Трактор", "City": "Минск"}]'
-----------------------------------------------------------------------------------

create procedure dbo.usp_GetSwimmersList
    @parameters nvarchar(1000) = null
as    
begin
    select  s.FirstName,
            s.LastName,
            s.YearOfBirth,
            ISNULL(sc.[Name] + ' ' + sc.City, '-') Club,
            ISNULL(c.[Name], '-') Category
    from dbo.Swimmer s
    left join dbo.SwimmingClub sc   on s.SwimmingClubID = sc.SwimmingClubID
    left join dbo.Category c        on s.CategoryID = c.CategoryID
    outer apply (
        select  [Name], 
				City
        from openjson(@parameters) 
            with (
                     [Name] nvarchar(100) '$.Club',
                     City nvarchar(100) '$.City'
                 ) j
        where (j.[Name] = sc.[Name] or j.[Name] is null) and
              (j.City = sc.City or j.City is null)
    ) f
    where (f.City is not null or f.[Name] is not null) or @parameters is null
end
go

<img src ="img/SwimmersDB-LogicalModel.svg" />